# Using Noise by Perturbing Each Image in a Certain Region

In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
import random

class RegionalNoise():
    # returns the CIFAR-10 data set (info here: https://keras.io/api/datasets/cifar10/)
    # The data set loaded is has 50,000 training images and 10,000 testing images, but the function below only takes
    # a certain number train_size of training images from the original training set. The rest of the training examples
    # are added to the test images.
    def generate_data(train_size):
        # loads the data in 50000,10000 form with labels (y) as integers, not one-hot
        (x_train_bef, y_train_bef), (x_test_bef, y_test_bef) = tf.keras.datasets.cifar10.load_data()

        # shuffles 50000 training data
        np.random.shuffle(x_train_bef)
        np.random.shuffle(y_train_bef)

        # cuts off train_size amounts of sample data
        x_train = x_train_bef[:train_size]
        y_train = y_train_bef[:train_size]

        # adds the rest of the training data to the test data
        x_test = np.concatenate((x_train_bef[train_size:], x_test_bef), axis=0)
        y_test = np.concatenate((y_train_bef[train_size:], y_test_bef), axis=0)

        # turns image arrays into floats just so that everything is a float, not an int
        x_train = x_train.astype(np.float)
        x_test = x_test.astype(np.float)

        # turns label data into one-hot form
        y_train = tf.keras.utils.to_categorical(y_train)
        y_test = tf.keras.utils.to_categorical(y_test)

        return x_train, y_train, x_test, y_test

    # looks at the training data, clones it, but then creates small changes to each image
    # Currently, it takes 3 random 5x5 sections of each image and perturbs each section.
    # the input it original training data which is not changed, but copied and then perturbed: the perturbed result
    # is returned
    def create_perturbed_clone(x_train, y_train):
        # new objects that are copied: changing x_train_pert won't change x_train
        x_train_pert = x_train.copy()
        y_train_pert = y_train.copy()

        # for each image in x_train_pert we look at a random location and perturb the 5x5 region
        length=x_train_pert.shape[0]
        for i in range(length):
            # perturbation is done 3 times
            for repeat in range(3):
                loc_x = random.randrange(2,30) # x-coord of center of 5x5 region
                loc_y = random.randrange(2,30) # y-coord of center of 5x5 region

                # each pixel and each RGB value is perturbed
                for u in [-2,-1,0,1,2]:
                    for v in [-2,-1,0,1,2]:
                        x_train_pert[i][loc_x+u][loc_y+v][0] += 50*(random.random()-0.5)
                        x_train_pert[i][loc_x+u][loc_y+v][1] += 50*(random.random()-0.5)
                        x_train_pert[i][loc_x+u][loc_y+v][2] += 50*(random.random()-0.5)

        return x_train_pert, y_train_pert

    # takes training data and adds a perturbed copy to make the training set enlarge_factor times larger
    def create_noise(x_train, y_train, enlarge_factor):
        x_train_noisy = x_train.copy()
        y_train_noisy = y_train.copy()
        for i in range(enlarge_factor-1):
            x_add, y_add = RegionalNoise.create_perturbed_clone(x_train, y_train)
            x_train_noisy = np.concatenate((x_train_noisy, x_add))
            y_train_noisy = np.concatenate((y_train_noisy, y_add))
        return x_train_noisy, y_train_noisy

    # creates network: we use a convolutional neural network which makes sense for this problem
    def create_model():
        model = Sequential()
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(10, activation='softmax')) # output lyer is 10-dimension one-hot, so softmax is used

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    def run_model(Epochs, BatchSize, trainSize, Set_enlarge, noisy=False):
        x_train, y_train, x_test, y_test = RegionalNoise.generate_data(trainSize)
        if noisy:
            x_train, y_train = RegionalNoise.create_noise(x_train, y_train, Set_enlarge)
        MODEL = RegionalNoise.create_model()
        MODEL.fit(x=x_train, y=y_train, epochs=Epochs, batch_size=BatchSize)
        MODEL.evaluate(x_test,y_test)

In [12]:
# Whether with or without noise, the model is accurate about 0.1 of the time on the test data, i.e. it is no
# better than a random guesser.
RegionalNoise.run_model(8,64,6000, noisy=True, Set_enlarge=15)

NameError: name 'create_noise' is not defined

# Using Noise by Perturbing Each Pixel with Equal Probability (Salt and Pepper)

In [2]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
import random

class Salt_and_Pepper():
    # returns the CIFAR-10 data set (info here: https://keras.io/api/datasets/cifar10/)
    # The data set loaded is has 50,000 training images and 10,000 testing images, but the function below only takes
    # a certain number train_size of training images from the original training set. The rest of the training examples
    # are added to the test images.
    def generate_data(train_size):
        # loads the data in 50000,10000 form with labels (y) as integers, not one-hot
        (x_train_bef, y_train_bef), (x_test_bef, y_test_bef) = tf.keras.datasets.cifar10.load_data()

        # shuffles 50000 training data
        np.random.shuffle(x_train_bef)
        np.random.shuffle(y_train_bef)

        # cuts off train_size amounts of sample data
        x_train = x_train_bef[:train_size]
        y_train = y_train_bef[:train_size]

        # adds the rest of the training data to the test data
        x_test = np.concatenate((x_train_bef[train_size:], x_test_bef), axis=0)
        y_test = np.concatenate((y_train_bef[train_size:], y_test_bef), axis=0)

        # turns image arrays into floats just so that everything is a float, not an int
        x_train = x_train.astype(np.float)
        x_test = x_test.astype(np.float)

        # turns label data into one-hot form
        y_train = tf.keras.utils.to_categorical(y_train)
        y_test = tf.keras.utils.to_categorical(y_test)

        return x_train, y_train, x_test, y_test

    # looks at the training data, clones it, but then creates small changes to each image
    # Currently, it perturbs a pixel with probability 0.1 (and changes each RGB value by a random amount)
    # the input is original training data which is not changed, but copied and then perturbed: the perturbed result
    # is returned
    def create_perturbed_clone(x_train, y_train):
        # new objects that are copied: changing x_train_pert won't change x_train
        x_train_pert = x_train.copy()
        y_train_pert = y_train.copy()

        # for each image in x_train_pert we perturb each pixel with probability 0.1
        shape=x_train_pert.shape
        for i in range(shape[0]):
            for x in range(shape[1]):
                for y in range(shape[2]):
                    num = random.random()
                    if num<0.1:
                        x_train_pert[i][x][y][0] += 100*(random.random()-0.5)
                        x_train_pert[i][x][y][1] += 100*(random.random()-0.5)
                        x_train_pert[i][x][y][2] += 100*(random.random()-0.5)

        return x_train_pert, y_train_pert

    # takes training data and adds a perturbed copy to make the training set enlarge_factor times larger
    def create_noise(x_train, y_train, enlarge_factor):
        x_train_noisy = x_train.copy()
        y_train_noisy = y_train.copy()
        for i in range(enlarge_factor-1):
            x_add, y_add = Salt_and_Pepper.create_perturbed_clone(x_train, y_train)
            x_train_noisy = np.concatenate((x_train_noisy, x_add))
            y_train_noisy = np.concatenate((y_train_noisy, y_add))
        return x_train_noisy, y_train_noisy

    # creates network: we use a convolutional neural network which makes sense for this problem
    def create_model():
        model = Sequential()
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(10, activation='softmax')) # output lyer is 10-dimension one-hot, so softmax is used

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    def run_model(Epochs, BatchSize, trainSize, Set_enlarge, noisy=False):
        x_train, y_train, x_test, y_test = Salt_and_Pepper.generate_data(trainSize)
        if noisy:
            x_train, y_train = Salt_and_Pepper.create_noise(x_train, y_train, Set_enlarge)
        MODEL = Salt_and_Pepper.create_model()
        MODEL.fit(x=x_train, y=y_train, epochs=Epochs, batch_size=BatchSize)
        MODEL.evaluate(x_test,y_test)

In [22]:
# Again, not much effect
Salt_and_Pepper.run_model(4,64,6000, noisy=True, Set_enlarge=10)

Epoch 1/4
938/938 [==============================] - 96s 99ms/step - loss: 4.3992 - accuracy: 0.1427
Epoch 2/4
938/938 [==============================] - 95s 101ms/step - loss: 2.1106 - accuracy: 0.5584
Epoch 3/4
938/938 [==============================] - 100s 107ms/step - loss: 1.5498 - accuracy: 0.8324
Epoch 4/4
1688/1688 [==============================] - 29s 17ms/step - loss: 7.8101 - accuracy: 0.0998


# Adding Gaussian Noise to Each Pixel

In [3]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
import random

class GaussianEvery():
    # returns the CIFAR-10 data set (info here: https://keras.io/api/datasets/cifar10/)
    # The data set loaded is has 50,000 training images and 10,000 testing images, but the function below only takes
    # a certain number train_size of training images from the original training set. The rest of the training examples
    # are added to the test images.
    def generate_data(train_size):
        # loads the data in 50000,10000 form with labels (y) as integers, not one-hot
        (x_train_bef, y_train_bef), (x_test_bef, y_test_bef) = tf.keras.datasets.cifar10.load_data()

        # shuffles 50000 training data
        np.random.shuffle(x_train_bef)
        np.random.shuffle(y_train_bef)

        # cuts off train_size amounts of sample data
        x_train = x_train_bef[:train_size]
        y_train = y_train_bef[:train_size]

        # adds the rest of the training data to the test data
        x_test = np.concatenate((x_train_bef[train_size:], x_test_bef), axis=0)
        y_test = np.concatenate((y_train_bef[train_size:], y_test_bef), axis=0)

        # turns image arrays into floats just so that everything is a float, not an int
        x_train = x_train.astype(np.float)
        x_test = x_test.astype(np.float)

        # turns label data into one-hot form
        y_train = tf.keras.utils.to_categorical(y_train)
        y_test = tf.keras.utils.to_categorical(y_test)

        return x_train, y_train, x_test, y_test

    # looks at the training data, clones it, but then creates small changes to each image
    # Currently, it adds a random amount (that varies according to Gaussian distribution) to each RGB value of each
    # pixel
    # the input is original training data which is not changed, but copied and then perturbed: the perturbed result
    # is returned
    def create_perturbed_clone(x_train, y_train):
        # new objects that are copied: changing x_train_pert won't change x_train
        x_train_pert = x_train.copy()
        y_train_pert = y_train.copy()

        # for each image in x_train_pert and each pixel of the image we perturb the RGB values by a Gaussian amount
        shape=x_train_pert.shape
        for i in range(shape[0]):
            for x in range(shape[1]):
                for y in range(shape[2]):
                    x_train_pert[i][x][y][0] += random.gauss(0,20)
                    x_train_pert[i][x][y][1] += random.gauss(0,20)
                    x_train_pert[i][x][y][2] += random.gauss(0,20)

        return x_train_pert, y_train_pert

    # takes training data and adds a perturbed copy to make the training set enlarge_factor times larger
    def create_noise(x_train, y_train, enlarge_factor):
        x_train_noisy = x_train.copy()
        y_train_noisy = y_train.copy()
        for i in range(enlarge_factor-1):
            x_add, y_add = GaussianEvery.create_perturbed_clone(x_train, y_train)
            x_train_noisy = np.concatenate((x_train_noisy, x_add))
            y_train_noisy = np.concatenate((y_train_noisy, y_add))
        return x_train_noisy, y_train_noisy

    # creates network: we use a convolutional neural network which makes sense for this problem
    def create_model():
        model = Sequential()
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(10, activation='softmax')) # output lyer is 10-dimension one-hot, so softmax is used

        
        #opt = keras.optimizers.SGD(learning_rate = 0.1, momentum = 0.9)
        #model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    def run_model(Epochs, BatchSize, trainSize, Set_enlarge, noisy=False):
        x_train, y_train, x_test, y_test = GaussianEvery.generate_data(trainSize)
        if noisy:
            x_train, y_train = GaussianEvery.create_noise(x_train, y_train, Set_enlarge)
        MODEL = GaussianEvery.create_model()
        MODEL.fit(x=x_train, y=y_train, epochs=Epochs, batch_size=BatchSize)
        MODEL.evaluate(x_test,y_test)

In [24]:
GaussianEvery.run_model(5, 64, 6000, Set_enlarge=10, noisy=True)

Epoch 1/5
938/938 [==============================] - 111s 116ms/step - loss: 5.2256 - accuracy: 0.1770
Epoch 2/5
938/938 [==============================] - 116s 124ms/step - loss: 1.9546 - accuracy: 0.6954
Epoch 3/5
938/938 [==============================] - 116s 124ms/step - loss: 1.4934 - accuracy: 0.8723
Epoch 4/5
938/938 [==============================] - 116s 124ms/step - loss: 1.3616 - accuracy: 0.9004
Epoch 5/5
1688/1688 [==============================] - 33s 19ms/step - loss: 7.8034 - accuracy: 0.1002


In [8]:
GaussianEvery.run_model(Epochs=2, BatchSize=128, trainSize=15000, Set_enlarge=10, noisy=True)

MemoryError: Unable to allocate 703. MiB for an array with shape (30000, 32, 32, 3) and data type float64

In [6]:
Salt_and_Pepper.run_model(Epochs=2, BatchSize=128, trainSize=15000, Set_enlarge=10, noisy=True)

Epoch 1/2
1172/1172 [==============================] - 717s 609ms/step - loss: 5.5415 - accuracy: 0.1246
Epoch 2/2
1407/1407 [==============================] - 70s 49ms/step - loss: 4.8277 - accuracy: 0.0986


# Summary

The neural net is unable to generalize off of 6,000 training samples (54,000 testing) in general. This is also the case when we add the three types of noise from above.

We now investigate if we increase trainin size and see what noise does. We change the architecture of the network a little to account for this (still using relu activation and a cnn, however).

Architecture:

        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(10, activation='softmax')) # output lyer is 10-dimension one-hot, so softmax is used


15,000 train, 45,000 test: 4 epochs, batch size 128, noise multiplies training set by 10
    
Without Noise: (12 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|3.1619|0.1016|2.4222|0.3392
2|2.8962|0.0966|2.4348|0.2762
3|3.4867|0.0999|2.3478|0.3993
4|2.8908|0.1022|2.4825|0.2516
5|2.9787|0.0966|2.3811|0.3023

Without Noise: (40 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|7.1343|0.0982|1.7263|0.8225

With Regional Noise: (2 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|5.0367|0.0994|2.5729|0.4287
2|4.9546|0.0970|2.4516|0.4538

With Salt and Pepper Noise: (2 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|4.1155|0.1032|2.7049|0.2644
2|4.8277|0.0986|2.6190|0.3976

With Every Pixel Gaussian Noise: (2 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-

Normal 50,000 training:

Architecture:

        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(10, activation='softmax'))
        
losses: 2.35, 2.4 (training, val); 
accuracy: 0.146, 0.1139

Similar to previous architecture's results 

This is likely significant (i.e. our net does stuff with lots of training)